# Project

In [1]:
import os
import pytimeloop.timeloopfe.v4 as tl

In [2]:
def run_timeloop_model(architecture, mapping, problem):
    if os.path.exists("./output_dir"):
        os.system("rm -r ./output_dir")

    spec = tl.Specification.from_yaml_files(architecture, mapping, problem)
    
    return tl.call_model(spec, output_dir="./output_dir")

In [3]:
result = run_timeloop_model("./test/arch.yaml", "./test/map.yaml", "./test/prob.yaml")
stats = open('./output_dir/timeloop-model.stats.txt', 'r').read()
print(stats)

Buffer and Arithmetic Levels
----------------------------
Level 0
-------
=== MACC ===

    SPECS
    -----
    Word bits             : 16
    Instances             : 1 (1*1)
    Compute energy        : 0.84 pJ

    STATS
    -----
    Utilized instances      : 1
    Computes (total)        : 48
    Cycles                  : 48
    Energy (total)          : 40.56 pJ
    Area (total)            : 466.38 um^2

Level 1
-------
=== Buffer ===

    SPECS
    -----
        Technology                      : SRAM
        Size                            : 64
        Word bits                       : 8
        Block size                      : 1
        Cluster size                    : 1
        Instances                       : 1 (1*1)
        Shared bandwidth                : -
        Read bandwidth                  : -
        Write bandwidth                 : -
        Multiple buffering              : 1.00
        Effective size                  : 64
        Min utilization               

In [ ]:
result = run_timeloop_model("./test/arch.yaml", "./test/map.yaml", "./test/prob.yaml")